# Curating a Dataset From the ChEMBL Database

In this notebook, we'll download and format data from the [ChEMBL](https://www.ebi.ac.uk/chembl/) database for machine learning (ML) purposes. To begin, we need to identify the **ChEMBL ID** for our target of interest. This is often easiest to do by using the **UniProt ID**, which can typically be found from a [UniProt Website](https://www.uniprot.org/) or looking at a relevant protein structure in the PDB. For this notebook, we will download data for [Epidermal Growth Factor Receptor (EGFR)](https://www.uniprot.org/uniprotkb/P00533/entry), a protein that plays a significant role in cancer.

Our task is made easier by the [chembl-downloader](https://github.com/cthoyt/chembl-downloader) package written Charles Tapley-Hoyt. This library simplifies the process of downloading and querying the ChEMBL database. For more on the chEMBL-downloader please see Charlie's [paper](https://joss.theoj.org/papers/10.21105/joss.08844) in The Journal of Open Source Software.

Install the necessary Python libraries

In [15]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !uv pip install --system pandas chembl_downloader

Using Python 3.12.11 environment at: /usr
Audited 2 packages in 248ms


Import the libraries we will use
- chembl_downloader - allows easy downloads and access to the ChEMBL database
- pandas - enables access to data tables
- sys - provides access to system functionality
- files - allows downloads from Google Colab

In [3]:
import chembl_downloader
import pandas as pd
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  from google.colab import files

To begin, we will use the chembl-downloader to download a file containing the mapping of UniProt IDs to ChEMBL IDs. Once we've downloaded the file we can look up the ChEMBL ID corresponding to the UniProt ID for EGFR.

In [17]:
uniprot_path = chembl_downloader.download_uniprot_mapping()
uniprot_df = pd.read_csv(uniprot_path,sep="\t",names=["UniProt", "ChEMBL_ID","Name","Target_Type"],comment="#")
uniprot_df.query("UniProt == 'P00533' and Target_Type == 'SINGLE PROTEIN'")

,UniProt,ChEMBL_ID,Name,Target_Type
306,P00533,CHEMBL203,Epidermal growth factor receptor erbB1,SINGLE PROTEIN


To access the data corresponding to our target of interest, we need to use a relatively complex SQL query.  I borrowed thiis query from the [OpenADMET](https://github.com/OpenADMET/openadmet-toolkit/blob/main/openadmet/toolkit/database/chembl.py) data curation workflow. The function below encapsulates this query.  We provide the ChEMBL ID as input and the fuction returns the sql query.

In [5]:
def get_query(chembl_id):
  query =f"""       -- Get all the activity data for a given target using its ChEMBL ID.
        select
            activities.assay_id                  as assay_id,
            activities.doc_id                    as doc_id,
            activities.standard_value            as standard_value,
            molecule_hierarchy.parent_molregno   as molregno,
            compound_structures.canonical_smiles as canonical_smiles,
            compound_structures.standard_inchi_key as standard_inchi_key,
            target_dictionary.tid                as tid,
            target_dictionary.chembl_id          as target_chembl_id,
            pchembl_value                        as pchembl_value,
            molecule_dictionary.pref_name        as compound_name,
            activities.standard_type             as standard_type,
            activities.bao_endpoint              as bao_endpoint,
            assays.description                   as assay_description,
            assays.assay_organism                as assay_organism,
            assays.assay_strain                  as assay_strain,
            assays.assay_tissue                  as assay_tissue,
            assays.assay_type                    as assay_type,
            assays.assay_cell_type               as assay_cell_type,
            assays.assay_subcellular_fraction    as assay_subcellular_fraction,
            assays.variant_id                    as variant_id,
            docs.year                            as doc_year,
            docs.journal                         as doc_journal,
            docs.doi                             as doc_doi,
            docs.title                           as doc_title,
            docs.authors                         as doc_authors,
            docs.abstract                        as doc_abstract,
            docs.patent_id                       as doc_patent_id,
            docs.pubmed_id                       as doc_pubmed_id,
            docs.chembl_release_id               as doc_chembl_release_id
        from activities
        join assays ON activities.assay_id = assays.assay_id
        join target_dictionary ON assays.tid = target_dictionary.tid
        join target_components ON target_dictionary.tid = target_components.tid
        join component_class ON target_components.component_id = component_class.component_id
        join docs ON activities.doc_id = docs.doc_id
        join molecule_dictionary ON activities.molregno = molecule_dictionary.molregno
        join molecule_hierarchy ON molecule_dictionary.molregno = molecule_hierarchy.molregno
        join compound_structures ON molecule_hierarchy.parent_molregno = compound_structures.molregno
        where target_chembl_id = '{chembl_id}'
        and activities.data_validity_comment IS null
        and activities.standard_units = 'nM'
        and pchembl_value is not null"""
  return query

Use the function above to get the SQL query.

In [6]:
chembl_id = 'CHEMBL203'
sql = get_query(chembl_id)

Run the SQL query. The ChEMBL downloader will first check to see if you've downloaded the ChEMBL database.  If you haven't, it will download and unpack it for you.  On Google Colab this typically takes 5-10 minutes. Run this cell, then go get a cup of coffee.

Once the database is downloaded, it will run the query and return the result as a Pandas dataframe. We also write the results to a csv file.

In [7]:
%%time
df = chembl_downloader.query(sql)
df.to_csv(f"{chembl_id}.csv", sep='\t', index=False)

CPU times: user 3min 2s, sys: 59.2 s, total: 4min 1s
Wall time: 8min 28s


Let's look at how many of each of the result types are available in the data we downloaded.

In [10]:
df.standard_type.value_counts()

,count
standard_type,
IC50,13158
Kd,809
Ki,568
EC50,285
Potency,35
AC50,11


As we can see above we have much more IC50 data than any other datatype.  However, as a recent [paper](https://pubs.acs.org/doi/full/10.1021/acs.jcim.4c00049) from Greg Landrum and Sereina Rinker showed, Ki data tends to be more reliable.  Let's download the Ki data.  We will select the three columns critical for building our ML model.
- canonical_smiles - the SMILES representation for the chemical structure
- molregno - the internal ChEMBL ID for the compounds
- pchembl_value - the negative log of the activity

In [11]:
standard_type = "Ki"
output_df = df.query("standard_type == @standard_type")[["canonical_smiles","molregno","pchembl_value"]].groupby(["canonical_smiles","molregno"]).mean()
output_df = output_df.reset_index()
output_df

,canonical_smiles,molregno,pchembl_value
0,C#CCCn1cc(Nc2nc(OC[C@H]3CN(C(=O)C=C)C[C@@H]3OC...,2076850,8.000000
1,C#CCOc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1NC(=O)C=C,2076849,8.700000
2,C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1,14785,8.547692
3,C=CC(=O)N1CC[C@@H](COc2nc(Nc3cnn(C)c3)nc3[nH]c...,2076841,8.100000
4,C=CC(=O)N1CC[C@@H](COc2nc(Nc3cnn(C)c3)nc3[nH]c...,2076839,6.230000
...,...,...,...
384,c1ccc(CNc2cc(-c3c[nH]c4ncccc34)ncn2)cc1,1323637,6.200000
385,c1ccc(CNc2ccnc3oc4ccccc4c23)cc1,716387,6.230000
386,c1ccc(COc2ccc(Nc3ncnc4ccccc34)cc2)cc1,1301454,7.000000
387,c1ccc(Nc2ncnc3ccccc23)cc1,55040,7.100000


Write the data to a csv file.

In [12]:
outfile_name = f"{chembl_id}_{standard_type}.csv"
output_df.to_csv(outfile_name, index=False)
print(f"Saved {outfile_name} with {len(output_df)} rows.")

Saved CHEMBL203_Ki.csv with 389 rows.


Let's take a quick look at the csv file.

In [13]:
!head {outfile_name}

canonical_smiles,molregno,pchembl_value
C#CCCn1cc(Nc2nc(OC[C@H]3CN(C(=O)C=C)C[C@@H]3OC)c3c(Cl)c[nH]c3n2)cn1,2076850,8.0
C#CCOc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1NC(=O)C=C,2076849,8.7
C#Cc1cccc(Nc2ncnc3cc(OCCOC)c(OCCOC)cc23)c1,14785,8.547692307692309
C=CC(=O)N1CC[C@@H](COc2nc(Nc3cnn(C)c3)nc3[nH]cc(Cl)c23)C1,2076841,8.1
C=CC(=O)N1CC[C@@H](COc2nc(Nc3cnn(C)c3)nc3[nH]ccc23)C1,2076839,6.23
C=CC(=O)N1CC[C@@H](Nc2nc(Nc3ccc(N4CCN(C)CC4)cc3)c3ncn(C(C)C)c3n2)C1,2284463,7.5
C=CC(=O)N1CC[C@@H](Nc2nc(Nc3cnn(C)c3)c3ncn(C(C)C)c3n2)C1,2287519,6.98
C=CC(=O)N1CC[C@@H](Nc2nc(Nc3cnn(C)c3)nc3[nH]cc(Cl)c23)C1,2284632,8.4
C=CC(=O)N1CC[C@H](Nc2nc(Nc3ccc(N4CCN(C)CC4)cc3)c3ncn(C(C)C)c3n2)C1,2291892,6.79


If you're on Google Colab, you can execute the cell below to download the results to your Downloads directory. If you're running the notebook locally the output file will be in the same directory as this notebook.

In [14]:
if IN_COLAB:
  files.download(outfile_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>